In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import zipfile
import shutil

# For Data Visualization
import cv2
import seaborn as sns

# For Model Building
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential, Model # Sequential API for sequential model
from tensorflow.keras.layers import Dense, Dropout, Flatten # Importing different layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Input, LeakyReLU, Activation
from tensorflow.keras import backend
from tensorflow.keras.utils import to_categorical # To perform one-hot encoding
from tensorflow.keras.optimizers import RMSprop, Adam, SGD # Optimizers for optimizing the model
from tensorflow.keras.callbacks import EarlyStopping  # Regularization method to prevent the overfitting
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import losses, optimizers
from tensorflow.keras.preprocessing.image import load_img
from google.colab.patches import cv2_imshow

In [2]:
flowers_root = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
import shutil

# Copy the dataset to /content so it's visible in the Files pane
shutil.copytree(flowers_root, '/content/flower_photos')

# Now the dataset will appear in the Files section

'/content/flower_photos'

In [4]:


# Now the dataset will appear in the Files section
count = 0
dirs = os.listdir('/content/flower_photos')

for dir in dirs:
    # Correctly join the paths
    folder_path = os.path.join('/content/flower_photos', dir)
    if os.path.isdir(folder_path):  # Ensure it is a directory
        files = os.listdir(folder_path)
        print(dir + ' Folder has ' + str(len(files)) + ' images')
        count += len(files)

print('Images Folder has ' + str(count) + ' images in total.')


sunflowers Folder has 699 images
daisy Folder has 633 images
dandelion Folder has 898 images
tulips Folder has 799 images
roses Folder has 641 images
Images Folder has 3670 images in total.


In [5]:
import os
import shutil
import random
import tensorflow as tf

# Define parameters
base_dir = '/content/flower_photos'  # Original directory containing subfolders
img_size = 180  # Image size
batch = 32  # Batch size for loading data
train_dir = '/content/flower_photos_train'  # Directory to store training data
test_dir = '/content/flower_photos_test'  # Directory to store testing data
split_ratio = 0.8  # 80% for training, 20% for testing

# Function to split and move the data into train and test directories
def move_data(base_dir, train_dir, test_dir, split_ratio=0.8):
    """
    Move the images from the source folder to train and test folders.

    Args:
        base_dir (str): The directory containing category subfolders (e.g., 'daisy', 'dandelion').
        train_dir (str): The directory to store training images.
        test_dir (str): The directory to store testing images.
        split_ratio (float): Ratio of data to use for training (default: 0.8 for training, 0.2 for testing).
    """
    # Loop through each class folder (e.g., 'daisy', 'dandelion')
    for class_name in os.listdir(base_dir):
        class_path = os.path.join(base_dir, class_name)

        # Skip if it's not a directory
        if not os.path.isdir(class_path):
            continue

        # Create subdirectories for each class in train and test directories
        train_class_dir = os.path.join(train_dir, class_name)
        test_class_dir = os.path.join(test_dir, class_name)

        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)

        # List all files in the class folder
        files = os.listdir(class_path)
        random.shuffle(files)  # Shuffle the files to ensure random splitting

        # Split files into training and testing sets
        split_index = int(len(files) * split_ratio)
        train_files = files[:split_index]
        test_files = files[split_index:]

        # Move files to the respective train and test subdirectories
        for file_name in train_files:
            shutil.copy(os.path.join(class_path, file_name), train_class_dir)
        for file_name in test_files:
            shutil.copy(os.path.join(class_path, file_name), test_class_dir)

# Move the data into train/test directories
move_data(base_dir, train_dir, test_dir, split_ratio)

# Load the data using image_dataset_from_directory
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=123,
    batch_size=batch,
    image_size=(img_size, img_size)
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=123,
    batch_size=batch,
    image_size=(img_size, img_size)
)

# Get class names from the training dataset
flower_names = train_ds.class_names
print("Class names:", flower_names)

# Check the structure of the datasets
print("\nTraining Data Structure:")
for class_name in os.listdir(train_dir):
    print(f"{class_name}: {len(os.listdir(os.path.join(train_dir, class_name)))} images")

print("\nTesting Data Structure:")
for class_name in os.listdir(test_dir):
    print(f"{class_name}: {len(os.listdir(os.path.join(test_dir, class_name)))} images")


Found 2934 files belonging to 5 classes.
Found 736 files belonging to 5 classes.
Class names: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

Training Data Structure:
sunflowers: 559 images
daisy: 506 images
dandelion: 718 images
tulips: 639 images
roses: 512 images

Testing Data Structure:
sunflowers: 140 images
daisy: 127 images
dandelion: 180 images
tulips: 160 images
roses: 129 images


In [6]:
from keras import layers
from keras import models

img_size = 180  # Ensure this matches the dataset
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),  # Dropout layer added here
    layers.Dense(512, activation='relu'),
    layers.Dense(5, activation='softmax')  # Use 5 because there are 5 flower classes
])




/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(
    loss='sparse_categorical_crossentropy',  # Use this for integer labels
    optimizer='adam',  # Adam optimizer as an alternative to RMSprop
    metrics=['accuracy']
)

In [8]:
history = model.fit(
    train_ds,  # Training dataset
    validation_data=val_ds,  # Validation dataset
    epochs=25,  # Number of epochs
    verbose=1
)


Epoch 1/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 279s 3s/step - accuracy: 0.2339 - loss: 20.1180 - val_accuracy: 0.2473 - val_loss: 1.5999
Epoch 2/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 284s 3s/step - accuracy: 0.2671 - loss: 1.5919 - val_accuracy: 0.2446 - val_loss: 1.5987
Epoch 3/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 316s 3s/step - accuracy: 0.2565 - loss: 1.5878 - val_accuracy: 0.2486 - val_loss: 1.5979
Epoch 4/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 332s 3s/step - accuracy: 0.2670 - loss: 1.5814 - val_accuracy: 0.2473 - val_loss: 1.6056
Epoch 5/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 312s 3s/step - accuracy: 0.2636 - loss: 1.5829 - val_accuracy: 0.2473 - val_loss: 1.6176
Epoch 6/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 286s 3s/step - accuracy: 0.2761 - loss: 1.5676 - val_accuracy: 0.2391 - val_loss: 1.6221
Epoch 7/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 308s 3s/step - accuracy: 0.2974 - loss: 1.5532 - val_accuracy: 0.2351 - val_loss: 1.6077
Epoch 8/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 332s 3s/step - accuracy: 0.2772 - loss: 1.5669 - val_accuracy: 0.2527 - 

In [9]:
def classify_images(image_path):
    input_image = tf.keras.utils.load_img(image_path, target_size=(180,180))
    input_image_array = tf.keras.utils.img_to_array(input_image)
    input_image_exp_dim = tf.expand_dims(input_image_array,0)

    predictions = model.predict(input_image_exp_dim)
    result = tf.nn.softmax(predictions[0])
    outcome = 'The Image belongs to ' + flower_names[np.argmax(result)] + ' with a score of '+ str(np.max(result)*100)
    return outcome

In [11]:
classify_images('/content/sunflower.jpeg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step


'The Image belongs to sunflowers with a score of 40.454769134521484'

In [13]:
classify_images('/content/Rose.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


'The Image belongs to roses with a score of 30.46633005142212'

In [14]:
classify_images('/content/tulip.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


'The Image belongs to tulips with a score of 25.87929666042328'